<a href="https://colab.research.google.com/github/afsharp/Bioinformatics_Microarray_Analysis/blob/main/lungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
# Load libraries
import pandas as pd
import numpy as np

In [ ]:
#col_list = ["GENDER", "AGE","SMOKING", 'YELLOW_FINGERS','ANXIETY', 'PEER_PRESSURE','CHRONIC DISEASE',	'FATIGUE',	'ALLERGY',	'WHEEZING',	'ALCOHOL CONSUMING'	,'COUGHING',	'SHORTNESS OF BREATH'	,'SWALLOWING DIFFICULTY',	'CHEST PAIN']
data = pd.read_csv("/content/lungcancer (1).csv")
data1 = data.copy()
target = data['LUNG_CANCER']
features = data
data.pop('LUNG_CANCER')
#features

**train-test split code**

In [ ]:
data_split = data1.sample(frac=1)

train_size = 0.5
train_end = int(len(data_split)*train_size)

data_train = data_split[:train_end]
data_test = data_split[train_end:]

data_train

** Calculating the entropy of the whole dataset**

In [134]:
def total_entropy(train, label, class_list):
    total_row = train.shape[0]
    total_entr = 0
    
    for i in class_list:
        total = train[train[label] == i].shape[0]
        total_entr = - (total/total_row)*np.log2(total/total_row) 
        total_entr += total_entr
    
    return total_entr

**Calculating the entropy for the filtered dataset**

In [135]:
def calc_entropy(features, label, class_list):
    class_count = features.shape[0]
    entropy = 0
    
    for i in class_list:
        label_class_count = features[features[label] == i].shape[0]  
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count  
            entropy_class = - probability_class * np.log2(probability_class)  
        entropy += entropy_class
    return entropy

**Calculating information gain for a feature**

In [136]:
def calc_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique()
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]  
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list)  
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy  
        
    return total_entropy(train_data, label, class_list) - feature_info 

**Finding feature with highest information gain**

In [137]:
def find_highest_information_gaine(train_data, label, class_list):
    feature_list = train_data.columns.drop(label)
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list: 
        feature_info_gain = calc_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain:  
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

**Adding a node to the tree**

In [138]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) 
    tree = {}  
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]  
        assigned_to_node = False  
        for c in class_list:  
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] 

            if class_count == count: 
                tree[feature_value] = c 
                train_data = train_data[train_data[feature_name] != feature_value] 
                assigned_to_node = True
        if not assigned_to_node:  
            tree[feature_value] = "?"  
            
    return tree, train_data

**Performing ID3 Algorithm and generating Tree**

In [139]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0:
        max_info_feature = find_highest_information_gaine(train_data, label, class_list) 
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) 
        next_root = None
        
        if prev_feature_value != None:
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else:
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()):
            if branch == "?":
                feature_value_data = train_data[train_data[max_info_feature] == node]
                make_tree(next_root, node, feature_value_data, label, class_list) 

**Finding unique classes of the label and Starting the algorithm**

In [140]:
def id3(train_data, label):
    train_data = train_data.copy() 
    tree = {} 
    class_list = train_data[label].unique() 
    make_tree(tree, None, train_data, label, class_list) 
    return tree

In [ ]:
tree = id3(data_train, 'LUNG_CANCER')
print(tree)

In [ ]:
tree = id3(data_test, 'LUNG_CANCER')
print(tree)